In [ ]:
from dotenv import load_dotenv
from pathlib import Path
import requests
from datetime import datetime, timezone, timedelta
from pymongo import MongoClient
from bson.objectid import ObjectId
import json, os, subprocess

env_path = Path('\\\\SUSMANSERVER\\Active Server Drive\\HomeAutomation') / '.env'
print(f'.env Loaded: {load_dotenv(dotenv_path=env_path)}')

In [ ]:
db_name = 'OAuth2_Manager'
collection_name = 'Active_OAuth2'
my_client = MongoClient('mongodb://susmanserver:27017',
                        username=os.getenv("OAuth2_Manager_Username"),
                        password=os.getenv("OAuth2_Manager_Password"),
                        authSource=db_name,
                        authMechanism='SCRAM-SHA-256')
my_db = my_client[db_name]
my_col = my_db[collection_name]

In [ ]:
token = my_col.find({'_id':ObjectId('5fd61e859532201850007cdf')})[0]

In [ ]:
url = f'https://smartdevicemanagement.googleapis.com/v1/enterprises/{os.getenv("Google_Nest_Prodect_ID")}/devices'
headers = {'Content-Type': 'application/json', 'Authorization': f'Bearer {token["access_token"]}'}
response = requests.get(url=url,headers=headers)

In [ ]:
name_type_dict = {}
response_json = json.loads(response.text)
for device in response_json['devices']:
    name_type_dict[device['name']] = device['type'].replace('.','_')

In [ ]:
my_client = MongoClient('mongodb://susmanserver:27017',
                        username=os.getenv("Google_Nest_Username"),
                        password=os.getenv("Google_Nest_Password"),
                        authSource='Google_Nest',
                        authMechanism='SCRAM-SHA-256')
my_db = my_client["Google_Nest"]
my_col = my_db["PubSub_Events"]

In [ ]:
missing_type_records = my_col.find({"resourceUpdate.type": {'$exists': True}}).sort('timestamp',-1)
missing_type_records[0]

In [ ]:
deviceID = missing_type_records[0]['resourceUpdate']['name']
eventID = missing_type_records[0]['resourceUpdate']['events']['sdm_devices_events_CameraSound_Sound']['eventId']
print(deviceID)
print(eventID)

In [ ]:
url = f'https://smartdevicemanagement.googleapis.com/v1/{deviceID}:executeCommand'
headers = {'Content-Type': 'application/json', 'Authorization': f'Bearer {token["access_token"]}'}
data = {"command" : "sdm.devices.commands.CameraEventImage.GenerateImage","params" : {"eventId" : eventID}}
response = requests.post(url=url,headers=headers,data=json.dumps(data))
print(response.text)

In [ ]:
for record in missing_type_records:
    my_query = {'_id': record['_id']}
    new_values = {"$set": {"resourceUpdate.type": name_type_dict[record['resourceUpdate']['name']]}}
    my_col.update_one(my_query, new_values)